1. Provide options which dataset/forest to load
2. Precompute maps and cable roads
3. Set A-Value, slope, etc.
4. Provide pre-computed optimized results
5. Editor
   - top layout
   - Pareto frontier
   - Current layout properties
6. Export? to exact positions of tree


In [1]:
# Package Imports
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

from shapely.geometry import LineString, Polygon, Point
from shapely.affinity import rotate, scale

from importlib import reload

# to load the interactive plotting version
# %matplotlib widget
# or regular
# %matplotlib inline
# %gui qt

# Importing my modules
from src.main import (
    data_loading,
    geometry_operations,
    geometry_utilities,
    cable_road_computation,
    mechanical_computations,
)

/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.6) or chardet (None)/charset_normalizer (2.1.0) doesn't match a supported version!
  warnings.warn(


In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
tree_df, forest_area_gdf, height_df = data_loading.load_bestand_and_forest(1)

ModuleNotFoundError: No module named 'dash'